# Synapse Labeling (with length threshold)

In [1]:
import datajoint as dj
import numpy as np
from scipy.spatial import KDTree
import time
from tqdm import tqdm
import ipyvolume.pylab as p3


In [2]:
dj.config['display.limit'] = 20
pinky = dj.create_virtual_module('pinky', 'microns_pinky')

Connecting celiib@10.28.0.34:3306


In [3]:
schema = dj.schema('microns_pinky')

@schema
class SynapseCompartmentLabelFinalThresholdRevised(dj.Manual):
    definition = """
    -> pinky.Synapse
    ---
    (postsynaptic_label)->pinky.LabelKey
    """ # (postsynaptic_label)->pinky.LabelKey : tinyint unsigned

In [4]:
pinky.CoarseLabelFinal()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as pinky.ProofreadLabel
3,648518346341371119,0.35,computer_Auto,celiib,2019-06-11 18:52:17,=BLOB=,=BLOB=
3,648518346349386137,0.35,computer_Auto,celiib,2019-06-11 18:53:38,=BLOB=,=BLOB=
3,648518346349470171,0.35,computer_Auto,celiib,2019-06-11 18:49:31,=BLOB=,=BLOB=
3,648518346349471156,0.35,computer_Auto,celiib,2019-06-12 13:53:20,=BLOB=,=BLOB=
3,648518346349471500,0.35,computer_Auto,celiib,2019-06-12 14:18:35,=BLOB=,=BLOB=
3,648518346349471562,0.35,computer_Auto,celiib,2019-06-12 14:13:32,=BLOB=,=BLOB=
3,648518346349471565,0.35,computer_Auto,celiib,2019-06-12 14:11:11,=BLOB=,=BLOB=
3,648518346349471910,0.35,computer_Auto,celiib,2019-06-12 14:02:23,=BLOB=,=BLOB=
3,648518346349472574,0.35,computer_Auto,celiib,2019-06-12 14:27:07,=BLOB=,=BLOB=
3,648518346349472601,0.35,computer_Auto,celiib,2019-06-12 14:28:33,=BLOB=,=BLOB=


In [5]:
segment_ids = pinky.CoarseLabelFinal.fetch('segment_id')
mesh_fetch = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh & pinky.CompartmentFinal.proj()).fetch()
label_fetch = pinky.CoarseLabelFinal.fetch()

meshes = dict()
labels = dict()
for segment in tqdm(mesh_fetch):
    meshes[segment[1]] = (segment[5], segment[6])
    
for segment in tqdm(label_fetch):
    labels[segment[1]] = (segment[6], segment[7])
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
#                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'presyn_x', 'presyn_y', 'presyn_z')).T
                                           & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x', 'postsyn_y', 'postsyn_z')).T

100%|██████████| 326/326 [00:42<00:00,  7.36it/s]


In [6]:
pinky.SynapseExclude()

segmentation segmentation id,synapse_id synapse index within the segmentation,criteria_id key by which to lookup the exclusion criteria description


In [7]:
matched_synapses = dict()
synapses_to_exclude = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude.append(dict(segmentation=3,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/326 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|          | 1/326 [00:00<02:34,  2.11it/s]
0it [00:00, ?it/s]
49it [00:00, 69198.95it/s]
0it [00:00, ?it/s]
  1%|          | 3/326 [00:01<02:21,  2.29it/s]
0it [00:00, ?it/s]
  1%|          | 4/326 [00:01<02:20,  2.29it/s]
0it [00:00, ?it/s]
  2%|▏         | 5/326 [00:02<02:19,  2.31it/s]
0it [00:00, ?it/s]
  2%|▏         | 6/326 [00:02<02:54,  1.83it/s]
0it [00:00, ?it/s]
  2%|▏         | 7/326 [00:04<04:21,  1.22it/s]
0it [00:00, ?it/s]
  2%|▏         | 8/326 [00:05<05:17,  1.00it/s]
0it [00:00, ?it/s]
  3%|▎         | 9/326 [00:06<05:23,  1.02s/it]
0it [00:00, ?it/s]
  3%|▎         | 10/326 [00:07<05:36,  1.06s/it]
0it [00:00, ?it/s]
  3%|▎         | 11/326 [00:08<05:29,  1.04s/it]
0it [00:00, ?it/s]
  4%|▎         | 12/326 [00:10<05:41,  1.09s/it]
0it [00:00, ?it/s]
  4%|▍         | 13/326 [00:10<04:23,  1.19it/s]
0it [00:00, ?it/s]
  4%|▍         | 14/326 [00:11<04:36,  1.13it/s]
0it [00:00, ?it/s]
  5%|▍         | 15

In [23]:
#pinky.ExclusionCriteria.insert1(dict(criteria_id=0,criteria_desc='Labeling pipeline'))

In [24]:
pinky.ExclusionCriteria()

criteria_id key by which to lookup the exclusion criteria description,criteria_desc criteria by which this segment was excluded (preferred to add name of excluder as well)
0,Labeling pipeline


In [21]:
for syn in synapses_to_exclude:
    syn["criteria_id"] = 0

In [25]:
#add to synapse exclude
pinky.SynapseExclude.insert(synapses_to_exclude,skip_duplicates=True)

In [26]:
#need to insert the synapse excludes into the table

In [27]:
pinky.SynapseCompartmentLabelFinalThresholdRevised()

segmentation segmentation id,synapse_id synapse index within the segmentation,postsynaptic_label numeric label of the compartment
3,835,2
3,2426,2
3,3848,2
3,5106,2
3,7656,2
3,8128,2
3,8372,2
3,8472,2
3,9439,2
3,9673,2


In [28]:
segmentation = pinky.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>12] = 1

start = time.time()
SynapseCompartmentLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 326/326 [00:00<00:00, 648.88it/s]


84.9232976436615


# Spine Synapse Labeler (with length threshold)

In [29]:
@schema
class SynapseSpineLabelFinalThresholdRevised(dj.Manual):
    definition = """
    # Denotes which synapses are also on Spine Heads, Spines, or Spine Necks
    -> pinky.SynapseCompartmentLabelFinalThresholdRevised
    ---
    (spine_label)->pinky.LabelKey
    """ # (spine_label)->pinky.LabelKey : tinyint unsigned

In [30]:
mesh_fetch = np.array((pinky.PymeshfixDecimatedExcitatoryStitchedMesh & pinky.CompartmentFinal.proj()).fetch('segment_id', 'vertices', 'triangles')).T
label_fetch = np.array(pinky.OverlayedSpineLabel.fetch('segment_id', 'vertices', 'triangles')).T

meshes = dict()
for segment_id, mesh_vertices, mesh_triangles in tqdm(mesh_fetch):
    meshes[segment_id] = (mesh_vertices, mesh_triangles)

labels = dict()
for segment_id, label_vertices, label_triangles in tqdm(label_fetch):
    labels[segment_id] = (label_vertices, label_triangles)
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x','postsyn_y', 'postsyn_z')).T

100%|██████████| 326/326 [00:41<00:00,  6.92it/s]


In [31]:
matched_synapses = dict()
synapses_to_exclude_spine = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude_spine.append(dict(segmentation=3,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/326 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|          | 1/326 [00:00<02:46,  1.96it/s]
0it [00:00, ?it/s]
49it [00:00, 68279.37it/s]
0it [00:00, ?it/s]
  1%|          | 3/326 [00:01<02:29,  2.16it/s]
0it [00:00, ?it/s]
  1%|          | 4/326 [00:01<02:27,  2.19it/s]
0it [00:00, ?it/s]
  2%|▏         | 5/326 [00:02<02:23,  2.23it/s]
0it [00:00, ?it/s]
  2%|▏         | 6/326 [00:02<02:59,  1.78it/s]
0it [00:00, ?it/s]
  2%|▏         | 7/326 [00:04<04:26,  1.20it/s]
0it [00:00, ?it/s]
  2%|▏         | 8/326 [00:05<05:23,  1.02s/it]
0it [00:00, ?it/s]
  3%|▎         | 9/326 [00:06<05:29,  1.04s/it]
0it [00:00, ?it/s]
  3%|▎         | 10/326 [00:08<05:42,  1.08s/it]
0it [00:00, ?it/s]
  3%|▎         | 11/326 [00:09<05:35,  1.06s/it]
0it [00:00, ?it/s]
  4%|▎         | 12/326 [00:10<05:43,  1.09s/it]
0it [00:00, ?it/s]
  4%|▍         | 13/326 [00:10<04:23,  1.19it/s]
0it [00:00, ?it/s]
  4%|▍         | 14/326 [00:11<04:36,  1.13it/s]
0it [00:00, ?it/s]
  5%|▍         | 15

In [33]:
for syn in synapses_to_exclude_spine:
    syn["criteria_id"] = 0

In [34]:
len(pinky.SynapseExclude())

43706

In [35]:
len(pinky.SynapseExclude()) #use this one for the future

43706

In [36]:
# FIX THE SYNAPSES TO EXCLUDE SO HAS REASON

In [39]:
pinky.SynapseExclude.insert(synapses_to_exclude_spine,skip_duplicates=True)


In [40]:
segmentation = pinky.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>15] = 1
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]<=15]
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]>12]

start = time.time()
SynapseSpineLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 326/326 [00:00<00:00, 682.35it/s]


42.846227169036865


## Visualization of Labeled Synapses (no labeled Spines yet)

In [41]:
SynapseSpineLabelFinalThresholdRevised()

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment
3,1,13
3,3,13
3,36,13
3,106,13
3,118,13
3,130,13
3,227,13
3,247,13
3,285,13
3,291,13


In [ ]:
# pinky.LabelKey()

In [ ]:
# pinky.ProofreadLabel & 'segment_id=648518346341393609'

In [ ]:
# label_keys = [( 0, 'not_labeled', 'no_color'), ( 1, 'label_removed', 'no_color'),
#                ( 2, 'Apical', 'blue'), ( 3, 'Basal', 'yellow'),
#                ( 4, 'Oblique', 'green'), ( 5, 'Soma', 'red'),
#                ( 6, 'Axon-Soma', 'aqua'), ( 7, 'Axon-Dendr', 'off blue'),
#                ( 8, 'Dendrite', 'purple'), ( 9, 'Distal', 'pink'),
#                (10, 'ignore Error', 'brown'), (11, 'Unlabelable', 'tan'),
#                (12, 'Cilia', 'orange'), (13, 'Spine Head (unsure if correct label)', 'red'),
#                (14, 'Spine (unsure if correct label)', 'blue'),
#                (15, 'Spine Neck (unsure if correct label)', 'green'),
#                (16, 'Bouton (unsure if correct label)', 'ignore')]
# segment_id = 648518346341393609
# synapse_label_types = np.unique(list(matched_synapses[segment_id].values()))
# label_matches = np.array(list(matched_synapses[segment_id].items())).T
# synapses_by_compartment = dict()
# synapse_centroids_by_compartment = dict()
# for synapse_label_type in synapse_label_types:
#     synapses_by_compartment[synapse_label_type] = label_matches[0][np.where(label_matches[1]==synapse_label_type)[0]]
#     centroids = np.zeros((len(synapses_by_compartment[synapse_label_type]), 3), dtype=np.uint32)
#     for i, specific_synapse in enumerate(synapses_by_compartment[synapse_label_type]):
#         centroids[i] = mesh_synapses[segment_id].T[1:].T[mesh_synapses[segment_id].T[0]==specific_synapse]
#     synapse_centroids_by_compartment[synapse_label_type] = centroids
# p3.figure(width=1024, height=1024)
# vertices, triangles = meshes[segment_id]
# p3.plot_trisurf(*vertices.T/1000, triangles=triangles, color='black')
# for label_type, label_name, label_color in label_keys:
#     if label_type in synapse_label_types:
#         print("{} -> {} : {}".format(label_type, label_name, label_color))
#         p3.scatter(*(synapse_centroids_by_compartment[label_type] * [4, 4, 40]).T/1000, color=label_color, marker='sphere', size=0.1)
# p3.squarelim()
# p3.show()